Library


In [1]:
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import time
from difflib import SequenceMatcher

Chrome Options

In [2]:
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--window-size=1920,1080")

Get all link of all club

In [3]:
driver = webdriver.Chrome(options=chrome_options)
url = 'https://www.premierleague.com/clubs'
driver.get(url)
time.sleep(5)
clubs = driver.find_elements(By.CLASS_NAME, 'club-card-wrapper')
name_clubs = []
link_clubs = []

for club in clubs:
    name = club.find_element(By.CLASS_NAME, 'club-card__name').text
    name_clubs.append(name)
    link = club.find_element(By.TAG_NAME, 'a').get_attribute('href')
    link = link.replace("overview", "stats")
    link += "?se=719"
    link_clubs.append(link)

link_all_club = {
    "name_club": name_clubs,
    "link_club": link_clubs,
}

data_frame_link_all_club = pd.DataFrame(link_all_club, index=link_all_club['name_club'])
data_frame_link_all_club.to_excel('link_all_club.xlsx', index=False)


Get stats of all club

In [4]:
df_link = pd.read_excel('link_all_club.xlsx').copy()

df_link = pd.DataFrame(data=list(df_link['link_club']), index=list(df_link['name_club']))

all_data_of_clubs = []

for url in df_link.values:
    driver = webdriver.Chrome(options=chrome_options)
    driver.get(url[0])
    time.sleep(5)
    all_data_of_club = driver.find_elements(By.CLASS_NAME, "all-stats__regular-stat-container")
    name = driver.find_element(By.CLASS_NAME, 'club-header__team-name').text
    print(name)
    record = dict()
    for data in all_data_of_club:
        key = data.find_element(By.CLASS_NAME, 'all-stats__regular-stat-name').text
        value = data.find_element(By.CLASS_NAME, 'allStatContainer').text
        if value.__contains__("%"):
            value = value[:-1]
            record[key] = int(value)
        elif value.__contains__("."): 
            record[key] = float(value)
        elif value.__contains__(","):
            value = value.replace(",", "")
            record[key] = int(value)
        else:
            record[key] = int(value)
        
        print(key + ": " + value)
        
    all_data_of_clubs.append(record)
all_data_of_clubs_df = pd.DataFrame(all_data_of_clubs, index=link_all_club['name_club'])

Arsenal
Goals: 21
Goals per match: 1.75
Shots: 171
Shots on target: 61
Shooting accuracy %: 36
Penalties scored: 0
Big Chances Created: 32
Hit woodwork: 2
Passes: 5230
Passes per match: 435.83
Pass accuracy %: 85
Crosses: 215
Cross accuracy %: 22
Clean sheets: 4
Goals Conceded: 12
Goals conceded per match: 1.00
Saves: 34
Tackles: 203
Tackle success %: 64
Blocked shots: 48
Interceptions: 83
Clearances: 236
Headed Clearance: 120
Aerial Battles/Duels Won: 755
Errors leading to goal: 0
Own goals: 0
Yellow cards: 29
Red cards: 3
Fouls: 79
Offsides: 23
Aston Villa
Goals: 19
Goals per match: 1.58
Shots: 153
Shots on target: 48
Shooting accuracy %: 31
Penalties scored: 0
Big Chances Created: 34
Hit woodwork: 4
Passes: 5368
Passes per match: 447.33
Pass accuracy %: 86
Crosses: 192
Cross accuracy %: 25
Clean sheets: 1
Goals Conceded: 19
Goals conceded per match: 1.58
Saves: 31
Tackles: 191
Tackle success %: 59
Blocked shots: 42
Interceptions: 86
Clearances: 200
Headed Clearance: 104
Aerial Battl

Compare two name functions

In [5]:
from Levenshtein import ratio

def token_similarity(string1, string2):
    words1 = string1.lower().split()
    words2 = string2.lower().split()

    if not words1 or not words2:
        return 0.0 

    scores = []
    for word1 in words1:
        best_match = max([ratio(word1, word2) for word2 in words2], default=0)
        scores.append(best_match)

    return (sum(scores) / len(words1)) * 100 if words1 else 0

Get fixtures of all club

In [6]:
all_data_of_clubs_df['fixtures'] = None
count = -1
    
for url in df_link.values:
    driver = webdriver.Chrome(options=chrome_options)
    url_fixture = url[0].replace("stats?se=719", "fixtures")
    driver.get(url_fixture)
    time.sleep(5)
    
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);") 
        time.sleep(2)

        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    elements = driver.find_elements(By.CLASS_NAME, 'fixtures__date-container')
    competitors = []
    count+=1
    for element in elements:
        all_name = element.find_elements(By.CLASS_NAME, 'match-fixture__short-name')
        
        competitor = all_name[0].text if token_similarity(link_all_club['name_club'][count], all_name[0].text) < token_similarity(link_all_club['name_club'][count], all_name[1].text) else all_name[1].text
        competitors.append(competitor)
        
    print(link_all_club['name_club'][count])
    print(competitors)
        
    competitors_string = ", ".join(competitors)
    
    all_data_of_clubs_df.loc[link_all_club['name_club'][count], "fixtures"] = competitors_string

    driver.quit()

Arsenal
['West Ham', 'Man Utd', 'Fulham', 'Everton', 'Crystal Palace', 'Ipswich', 'Brentford', 'Brighton', 'Spurs', 'Aston Villa', 'Wolves', 'Man City', 'Leicester', 'West Ham', "Nott'm Forest", 'Man Utd', 'Chelsea', 'Fulham', 'Everton', 'Brentford', 'Ipswich', 'Crystal Palace', 'Bournemouth', 'Liverpool', 'Newcastle', 'Southampton']
Aston Villa
['Chelsea', 'Brentford', 'Southampton', "Nott'm Forest", 'Man City', 'Newcastle', 'Brighton', 'Leicester', 'Everton', 'Arsenal', 'West Ham', 'Wolves', 'Ipswich', 'Chelsea', 'Crystal Palace', 'Brentford', 'Liverpool', 'Brighton', "Nott'm Forest", 'Southampton', 'Newcastle', 'Man City', 'Fulham', 'Bournemouth', 'Spurs', 'Man Utd']
Bournemouth
['Wolves', 'Spurs', 'Ipswich', 'West Ham', 'Man Utd', 'Crystal Palace', 'Fulham', 'Everton', 'Chelsea', 'Newcastle', "Nott'm Forest", 'Liverpool', 'Southampton', 'Wolves', 'Brighton', 'Spurs', 'Brentford', 'Ipswich', 'West Ham', 'Fulham', 'Crystal Palace', 'Man Utd', 'Arsenal', 'Aston Villa', 'Man City', 'Le

Get rank of all club

In [9]:
url = 'https://www.premierleague.com/tables'
driver = webdriver.Chrome(options=chrome_options)
driver.get(url)
time.sleep(5)

last_height = driver.execute_script("return document.body.scrollHeight")
while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);") 
    time.sleep(2)

    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height
all_data_of_clubs_df['rank'] = None
all_data_of_clubs_df['points'] = None
all_data_of_clubs_df['performances'] = None
elements = driver.find_elements(By.XPATH, "//*[@class=' ' or @class='tableDark EU_CL' or @class='tableMid EU_CL' or @class='tableMid EU_UC' or @class='tableMid EU_CF']")

for element in elements:
    rank = element.find_element(By.CLASS_NAME, 'league-table__value').text
    name = element.find_element(By.CLASS_NAME, 'league-table__team-name').text
    if name == "Brighton And Hove Albion":
        name = "Brighton & Hove Albion"
    print(name)
    points = element.find_element(By.CLASS_NAME, 'league-table__points').text
    performances_html = element.find_elements(By.CLASS_NAME, 'form-abbreviation')
    performances = []
    for performance in performances_html:
        performances.append(performance.text)
    all_data_of_clubs_df.loc[name, "rank"] = int(rank)
    all_data_of_clubs_df.loc[name, "points"] = int(points)
    all_data_of_clubs_df.loc[name, "performances"] = ", ".join(performances)
    
all_data_of_clubs_df.to_excel("all_data_of_clubs.xlsx", index=True)

Liverpool
Manchester City
Chelsea
Arsenal
Brighton & Hove Albion
Tottenham Hotspur
Nottingham Forest
Aston Villa
Newcastle United
Fulham
Brentford
Manchester United
Bournemouth
West Ham United
Everton
Leicester City
Wolverhampton Wanderers
Crystal Palace
Ipswich Town
Southampton
